In [1]:
import os
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt

# Stage 1 : Data Ingestion

- Identify raw data
- Load it into an appropriate format for EDA

### Download Raw Data

In [2]:
# here is our raw data, ingestion will simply download from a hosted location
!wget -nc https://archive.ics.uci.edu/ml/machine-learning-databases/census1990-mld/USCensus1990.data.txt
!wget -nc https://archive.ics.uci.edu/ml/machine-learning-databases/census1990-mld/USCensus1990raw.data.txt
!wget -nc https://archive.ics.uci.edu/ml/machine-learning-databases/census1990-mld/USCensus1990raw.attributes.txt

File ‘USCensus1990.data.txt’ already there; not retrieving.

File ‘USCensus1990raw.data.txt’ already there; not retrieving.

File ‘USCensus1990raw.attributes.txt’ already there; not retrieving.



### Load Raw Data

In [3]:
ROOT = r"./"

In [4]:
def _load_data(ROOT):
    
    with open(os.path.join(ROOT,'USCensus1990raw.attributes.txt'), 'r') as f:
        text = f.read()
    print(text)
    
    
    sep = '__________________________________________________________________________________'

    all_codes = {}
    for t in text.split(sep)[1:]:

        lines = t.split('\n')

        colname = lines[1][:10].rstrip(' ')
        lines = [l.lstrip(' ').rstrip(' ') for l in lines[2:] if (l !='') and not l.startswith('VAR:')]
        coding = {}
        for line in lines:
            l = [l for l in line.split(' ') if l != '']
            coding[l[0]] = " ".join(l[1:])

        all_codes[colname] = coding
        
        
    cols = text.split('__________________________________________________________________________________')
    len(cols)
    cols = [c.split('\n')[1] for c in cols[1:]]
    cols = [c for c in cols if c != '']
    frame = ""
    for c in cols:
        li = [c[:10].rstrip(' '), c[13:14], c[21:22],c[28:29], c[42:].replace(",", ".")]
        if len(li) !=5:
            print(li)
        frame += r",".join(li) + "\n"


    df_meta = pd.read_csv(io.StringIO(frame), sep=',', header=None, names=['Var', 'Type', 'Des', 'Len', 'Description'])
    
    return df_meta, all_codes

In [5]:
df_meta, all_codes = _load_data(ROOT)

                           U.S. DEPARTMENT OF COMMERCE
                                 BUREAU OF CENSUS

                          *** DATA EXTRACTION SYSTEM ***

  DOCUMENTATION OF: FILE CONTENTS

  FOR DATA COLLECTION: 'pums901p' - 1990 Decenial Census 1% PUMS - Persons Records

    VAR: = Variable Name
    TYP: = Variable Type     ( C = Categorical, N = Numeric Continuous )
    DES: = Designation       ( P = Primary Variable, X = Non-Primary )
    LEN: = Length            ( of the Variable in Characters )
    CAT: = Category          ( of the Variable )


VAR:        TYP:   DES:    LEN:   CAT:    VARIABLE/CATEGORY LABEL:
__________________________________________________________________________________
AAGE         C       X      1             Age Allocation Flag
                                  0       No
                                  1       Yes

VAR:        TYP:   DES:    LEN:   CAT:    VARIABLE/CATEGORY LABEL:
_______________________________________________________________

In [ ]:
df = pd.read_csv(os.path.join(ROOT,'USCensus1990raw.data.txt'), 
                 delimiter="\t", 
                 header=0, 
                 dtype=str)
df.columns = list(df_meta['Var'].values)

### Explore the basic data properties

- 2.5 M records
- Large feature set

In [ ]:
def describe_feature(x): 
    v = df_meta[df_meta['Var'] == x]['Description'].values
    if len(v) == 0:
        return ''
    else:
        return v[0]

In [ ]:
print(f"Records:  {df.shape[0]}")
print(f"Features: {df.shape[1]}")

# Stage 2 :  Exploratory Data Analysis (EDA)

Also data cleaning

### Features

In [ ]:
for x in list(df.columns):
    print(f"{x:<20} - {describe_feature(x)}")

### A. Data Cleaning Example

In [ ]:
all_vars = list(df_meta['Var'])

vars_with_flag = []
for var in df_meta['Var']:
    if 'A'+var in all_vars:
        vars_with_flag += [var]

# convert to integer
for v in vars_with_flag:
    df['A'+v] = df['A'+v].astype('int')
    
df['AGE'] = df['AGE'].astype('int')

In [ ]:
# how many of these fields are populated?
# df[['A'+ v for v in vars_with_flag]].sum(axis=0)

We notice that a lot of these fields have the same names... it appears that many features are simply booleans indictaing that another field is populated or not....

We would do this in a much simpler way today

In [ ]:
all_vars = list(df_meta['Var'])

vars_with_flag = []
for var in df_meta['Var']:
    if 'A'+var in all_vars:
        vars_with_flag += [var]

df = df.drop(['A' + v for v in vars_with_flag], axis=1)

In [ ]:
alloc_flags = [
          'AANCSTR1',
          'AANCSTR2',
          'AAUGMENT',
          'ABIRTHPL',
          'ALABOR',
          'AHISPAN',
          'AINDUSTR',
          'ALSTWRK',
          'AMIGSTAT',
          'AMOBLLIM',
          'AMOBLTY',
          'APERCARE',
          'APOWST',
          'ATRAVTME',
          'AWKS89',
          'ASERVPER',
          'AVETS1']

for x in alloc_flags:
    print(f"{x:<20} - {describe_feature(x)}")
    print(df[x].unique())
    
# binary cols
df = df.drop(alloc_flags, axis=1)

In [ ]:
# from the description we only need REARNING
df = df.drop(['INCOME1',
              'INCOME2',
              'INCOME3',
              'INCOME4',
              'INCOME5',
              'INCOME6',
              'INCOME7',
              'INCOME8',
              'RPINCOME'], axis=1)

In [ ]:
df['AGE']      = df['AGE'].astype(int)
df['REARNING'] = df['REARNING'].astype(float)
df['SEX']      = df["SEX"].astype(int)
df['FERTIL']   = df["FERTIL"].astype(int)
df['PWGT1']    = df['PWGT1'].astype(float)

# military years served
df['YRSSERV']    = df['YRSSERV'].astype(int)

In [ ]:
df.head(1)

In [ ]:
# cardinality
_ = plt.hist((df.nunique(axis=0).values))

In [ ]:
# Spoken languages

# remove <5 year olds, see all_codes['LANG1']
df = df[df['LANG1'] != '0']
df['multilingual'] = df['LANG1'].apply(lambda x: {'2':0, '1':1}[x])
df = df.drop('LANG1', axis=1)

# drop LANG2 since complex category
df = df.drop('LANG2', axis=1)

# ability to speak english
df = df[df['ENGLISH'] != '0']

In [ ]:
# Family

# own child means under 18, exclude these and drop column
df = df[df['ROWNCHLD'] == '0']
df = df.drop('ROWNCHLD', axis=1)

In [ ]:
# Jobs

df = df[df['CLASS'] != '0'] # child
df = df[df['CLASS'] != '9'] # unemployed

df['TRAVTIME'] = df['TRAVTIME'].astype('float') # travel time in minutes
df['HOURS']    = df['HOURS'].astype('float')    # hours worked last week
df['HOUR89']   = df['HOUR89'].astype('float')   # usual hours previous year

df['RLABOR'] = df['RLABOR'].apply(lambda x: x in ['1','2','4','5'])

df['DEPART'] = df['DEPART'].apply(lambda x: int(x[:2]) * 60 + int(x[2:])) # departure for work convert to minutes since midnight

df = df[df['POWSTATE'] != '00'] # doesn't work
df = df[df['POWSTATE'] != '99'] # remove works abroad

# turn to boolean did/did not work last week
df['WORKLWK'] = df['WORKLWK'].apply(lambda x: {'0':False,'1':True,'2':False}[x])


In [ ]:
describe_feature('HOUR89')

In [ ]:
# SERIALNO and POVERTY have very high cardinality but we don't know what they refer to, so drop.
df = df.drop('SERIALNO', axis=1)
df = df.drop('POVERTY', axis=1)

In [ ]:
# TODO
# df['OCCUP'].apply(lambda x: all_codes['OCCUP'][x])
# df['INDUSTRY'].apply(lambda x: all_codes['INDUSTRY'][x])

In [ ]:
# convert to text for easier reading
df['RACE']         = df['RACE'].apply(lambda x: all_codes['RACE'][x])
df['SEX']          = df['SEX'].apply(lambda x:  all_codes['SEX'][str(x)])
df['MARITAL']      = df['MARITAL'].apply(lambda x: all_codes['MARITAL'][x])
df['CLASS']        = df['CLASS'].apply(lambda x: all_codes['CLASS'][x])
df['YEARSCH_CAT']  = df['YEARSCH'].apply(lambda x: all_codes['YEARSCH'][x])
df['MEANS']        = df['MEANS'].apply(lambda x: all_codes['MEANS'][x])

# Location
df['MIGSTATE']     = df['POWSTATE'].apply(lambda x: all_codes['POWSTATE'][str(x).zfill(2)])
df['POWSTATE']     = df['POWSTATE'].apply(lambda x: all_codes['POWSTATE'][str(x).zfill(2)])
df['POB']          = df['POB'].apply(lambda x: all_codes['POB'][str(x).zfill(3)])
df['RPOB']         = df['RPOB'].apply(lambda x: all_codes['RPOB'][str(x)])

df['RSPOUSE']      = df['RSPOUSE'].apply(lambda x: all_codes['RSPOUSE'][str(x)])
df['RAGECHLD']     = df['RAGECHLD'].apply(lambda x: all_codes['RAGECHLD'][str(x)])

In [ ]:
for x in list(df.columns):
    try:
        print(f"{x:<20} - {describe_feature(x)}")
    except:
        print(x)

### An example of adjusting feature to reduce imbalance

In [ ]:
all_codes['CITIZEN']

In [ ]:
df.groupby('CITIZEN').count()['AGE']

In [ ]:
citizen_map = {
    '0':True,
    '1':True,
    '2':True,
    '3':True,
    '4':False
}
df['CITIZEN'] = df['CITIZEN'].apply(lambda x: citizen_map[x])

In [ ]:
# this class is now much more balanced
df.groupby('CITIZEN').count()['AGE']

In [ ]:
df['CLASS']

In [ ]:
# exploring this feature it isn't doing a lot
df = df.drop('LOOKING', axis=1)

In [ ]:
military_map = {
    '0':False,
    '1':True,
    '2':True,
    '3':True,
    '4':False
}
print(all_codes['MILITARY'])
df['MILITARY'] = df['MILITARY'].apply(lambda x: military_map[x])

### B. Data Quality

- Focus on features we care about

In [ ]:
describe_feature('AGE')

In [ ]:
# all the ages present
x = df['AGE'].unique()
x.sort()

# that's a bit fishy... there are definitely people aged over 90 
print(f"Min age {x.min()}, Max age {x.max()}")

# assertions are how we capture quality checks
assert np.unique(np.diff(x)) == 1


print(x)

In [ ]:
df.head(5)

In [ ]:
# ah... a deeper look at the data reveals an anomalously high point at 90... 
# it is likley that everyone aged over 90 was counted as being 90
_ = plt.hist(df['AGE'], bins=np.arange(0,100,1))

In [ ]:
# How many served in WWII? 4%
df[['AGE', 'WWII']].astype(int).groupby('WWII').count()

In [ ]:
df_ww2 = df[['AGE', 'WWII']].astype(int)

_ = plt.hist(df['AGE'], bins=np.arange(0,100,1), histtype='step')
_ = plt.hist(df_ww2[df_ww2['WWII'] == 1]['AGE'], bins=np.arange(0,100,1), color='r')

In [ ]:
# can we find other wars represented?
df_meta[df_meta['Description'].apply(lambda x: "served" in x.lower())]

In [ ]:
service_cols = ['WWII', 'VIETNAM', 'KOREAN']
colors = ['r', 'g', 'b']

In [ ]:

fig, ax = plt.subplots(1,1,figsize=(10,10))

for i, w in enumerate(service_cols):
    df_war = df[['AGE', w]].astype(int)
    _ = ax.hist(df_war[df_war[w] == 1]['AGE'], bins=np.arange(0,100,1), 
                 color=colors[i], label=w)
    

_ = ax.hist(df['AGE'], bins=np.arange(0,100,1), histtype='step')

ax.legend(frameon=False)


### Jobs

In [ ]:
from bs4 import BeautifulSoup
import re

In [ ]:
with open('USCensus1990raw.coding.htm', 'r') as f:
    html_text=f.read()
soup = BeautifulSoup(html_text, 'html.parser')

In [ ]:
lines = soup.body.pre.text.split('\n')
i_start = [i for i,l in enumerate(lines) if 'OCCUPATION' in l][0]
i_stop = [i for i,l in enumerate(lines) if 'PLACE OF BIRTH' in l][0]

occups = lines[i_start:i_stop-1]
occups = [line for line in occups if len(line)>1]

# remove wrapped lines
occups_2 = []
for o in occups:
    if o.startswith('   '):
        occups_2[-1] += o.strip()
    else:
        occups_2 += [o]
occups = occups_2
del occups_2

In [ ]:
# split the individual jobs from their groupings (it's all in the same list)

pattern = re.compile('[0-9][0-9][0-9]-[0-9][0-9][0-9] *')
occups_single = [o for o in occups if pattern.match(o) is None]
occups_group  = [o for o in occups if pattern.match(o) is not None]

df_jobs   = pd.DataFrame([[o[:4].strip(), o[4:].strip()] 
                          for o in occups_single[8:]], columns=['Code', 'Job']) 
df_groups = pd.DataFrame([[o[:3], o[4:7], o[7:].strip()] 
                          for o in occups_group], columns=['Code min', 'Code max', 'Job']) 
df_groups['range'] = df_groups['Code max'].astype(int) - df_groups['Code min'].astype(int)

In [ ]:
# how many codes are not present in the single job rows?

jobs = list(df_jobs['Code'].values)

jobs_not_in_list = []
for i in df['OCCUP'].unique():
    if str(i).zfill(3) not in jobs:
        jobs_not_in_list += [i]
        
print(f"{len(jobs_not_in_list)} out of {len(df['OCCUP'].unique())}")

In [ ]:
# identify summary groups by large ranges and capitalization

summary_group_list = [
 'MANAGERIAL AND PROFESSIONAL SPECIALTY OCCUPATIONS',        # 0-202
 'TECHNICAL, SALES, AND ADMINISTRATIVE SUPPORT OCCUPATIONS', # 203-402
 'SERVICE OCCUPATIONS',                                      # 403-472
 'PRECISION PRODUCTION, CRAFT, AND REPAIR OCCUPATIONS',      # 503-702
 'OPERATORS, FABRICATORS, AND LABORERS',                     # 703-902
 'MILITARY OCCUPATIONS (Includes only uniquely military    occupations.Other Armed Forces members are coded to civilian occupations.)', 
                                                             # 903-908
 'EXPERIENCED UNEMPLOYED NOT CLASSIFIED BY OCCUPATION'       # 909-999
]

df_groups.sort_values('range', ascending=False).head(50)

df_summary_groups = df_groups[df_groups['Job'].apply(lambda x: x in summary_group_list)]
df_summary_groups


In [ ]:
df_groups.sort_values('range', ascending=False).head(10)['Job'].values

In [ ]:
# add in the missing job codes using the lowest cardinality grouping
for i in range(1000):
    code = str(i).zfill(3)
    if df_jobs[df_jobs['Code'] == code].shape[0] ==0:
        # add code to frame with job cat..
        matches = df_groups[(df_groups['Code min'] <= code) & (df_groups['Code max'] >= code)]
        the_job = matches.sort_values('range').iloc[0]['Job']
        df_jobs = df_jobs.append({'Code':code, 'Job':the_job}, ignore_index=True)
df_jobs = df_jobs.sort_values('Code').reset_index().drop('index',axis=1)

In [ ]:
df_jobs.head(5)

In [ ]:
# remove the summary groups from the group frame
df_groups = df_groups[df_groups['Job'].apply(lambda x: x not in summary_group_list)]
df_groups

In [ ]:
# determine how many other groups each group contains
conts = {}
for i, row in df_groups.iterrows():
    r1 = int(row['Code min'])
    r2 = int(row['Code max'])
    conts[row['Job']] = []
    for j, rowj in df_groups.iterrows():
        if int(rowj['Code min']) >= r1 and int(rowj['Code max']) <= r2:
            conts[row['Job']] += [rowj['Job']]
major_group_list = [k for k,v in conts.items() if len(v)>5]

In [ ]:
for k,v in conts.items():
    print(k, len(v))

In [ ]:
major_group_list = [
    'Executive, Administrative, and Managerial Occupations',   # 0-42
    'Professional Specialty Occupations',                      # 43-202
    'Technicians and Related Support Occupations',             # 203-242
     'Sales Occupations',                                      # 243-302
    'Administrative Support Occupations, Including Clerical',  # 303-402
    
    'Protective Service Occupations',                          # 413-432
    'Service Occupations, Except Protective and Household',    # 433-472
    
     'Mechanics and Repairers',                                # 503-552
#      'Mechanics and Repairers, Except Supervisors'
     'Construction Trades',                                    # 553-612
#      'Construction Trades, Except Supervisors',
    
     'Precision Production Occupations',                       # 628-702
     'Machine Operators, Assemblers, and Inspectors',          # 703-802
#      'Machine Operators and Tenders, Except Precision',
     'Transportation and Material Moving Occupations',         # 803-863
     'Handlers, Equipment Cleaners, Helpers, and Laborers',    # 864-902
     'MILITARY OCCUPATIONS (Includes only uniquely military    occupations.Other Armed Forces members are coded to civilian occupations.)', 
                                                               # 903-908
     'EXPERIENCED UNEMPLOYED NOT CLASSIFIED BY OCCUPATION'     # 909-999
]

df_major_groups = df_groups[df_groups['Job'].apply(lambda x: x in major_group_list)]
df_major_groups

In [ ]:
df_groups[df_groups['Job'].apply(lambda x: x in major_group_list)]

In [ ]:
list(df_groups[df_groups['Job'].apply(lambda x: x in major_group_list)]['Job'])

In [ ]:
# define the mappings of code -> Group

# ======= JOBS   ======================
jobs_map = {row['Code']:row['Job'] for i,row in df_jobs.iterrows()}

# ======= SUMMARY GROUPS ==============
summary_group_map = {}
for i in range(1000):
    code = str(i).zfill(3)

    # add code to frame with job cat..
    matches = df_summary_groups[  (df_summary_groups['Code min'] <= code) 
                                & (df_summary_groups['Code max'] >= code)]
    assert matches.shape[0] <= 1
    
    if matches.shape[0] == 0:
        summary_group_map[code] = 'UNKNOWN'
    else:
        summary_group_map[code] = matches['Job'].values[0]
        

# ======= MAJOR GROUPS =================
major_group_map = {}
for i in range(1000):
    code = str(i).zfill(3)

    # add code to frame with job cat..
    matches = df_major_groups[  (df_major_groups['Code min'] <= code) 
                                & (df_major_groups['Code max'] >= code)]
    assert matches.shape[0] <= 1
    
    if matches.shape[0] == 0:
        major_group_map[code] = 'UNKNOWN'
    else:
        major_group_map[code] = matches['Job'].values[0]

In [ ]:
# apply the maps

df['OCCUP_JOB'] = df['OCCUP'].apply(lambda x: jobs_map[str(x).zfill(3)])
df['OCCUP_SUM'] = df['OCCUP'].apply(lambda x: summary_group_map[str(x).zfill(3)])
df['OCCUP_MAJ'] = df['OCCUP'].apply(lambda x: major_group_map[str(x).zfill(3)])

In [ ]:
# quality checks

print(f"JOB Percentage unknown : {100*sum(df['OCCUP_JOB'] =='UNKNOWN') / df.shape[0]:.2f}%")
print(f"SUM Percentage unknown : {100*sum(df['OCCUP_SUM'] =='UNKNOWN') / df.shape[0]:.2f}%")
print(f"MAJ Percentage unknown : {100*sum(df['OCCUP_MAJ'] =='UNKNOWN') / df.shape[0]:.2f}%")

### Industry

In [ ]:
lines = soup.body.pre.text.split('\n')
i_start = [i for i,l in enumerate(lines) if 'INDUSTRY' in l][0]
i_stop = [i for i,l in enumerate(lines) if 'LANGUAGE' in l][0]

industry = lines[i_start:i_stop-1]
industry = [line for line in industry if len(line)>1]

# remove wrapped lines
industry_2 = []
for o in industry:
    if o.startswith('   '):
        industry_2[-1] += o.strip()
    else:
        industry_2 += [o]
industry = industry_2
del industry_2

# cut away header
industry = industry[8:]

In [ ]:
industry

In [ ]:
# split the individual jobs from their groupings (it's all in the same list)

pattern = re.compile('[0-9][0-9][0-9]-[0-9][0-9][0-9] *')
occups_single = [o for o in industry  if pattern.match(o) is None]
occups_group  = [o for o in industry  if pattern.match(o) is not None]

df_jobs   = pd.DataFrame([[o[:4].strip(), o[4:].strip()] 
                          for o in occups_single[8:]], columns=['Code', 'Job']) 
df_groups = pd.DataFrame([[o[:3], o[4:7], o[7:].strip()] 
                          for o in occups_group], columns=['Code min', 'Code max', 'Job']) 
df_groups['range'] = df_groups['Code max'].astype(int) - df_groups['Code min'].astype(int)

In [ ]:
df_groups

In [ ]:
# how many codes are not present in the single job rows?

jobs = list(df_jobs['Code'].values)

jobs_not_in_list = []
for i in df['INDUSTRY'].unique():
    if str(i).zfill(3) not in jobs:
        jobs_not_in_list += [i]
        
print(f"{len(jobs_not_in_list)} out of {len(df['INDUSTRY'].unique())}")

In [ ]:
df_groups[df_groups['Code min'] == '040']

In [ ]:
df_groups.sort_values('range', ascending=False).head(25)

In [ ]:
# identify summary groups by large ranges and capitalization

summary_group_list = [
 'AGRICULTURE, FORESTRY, AND FISHERIES' ,                      # 000-039
 'MINING',                                                     # 040-059
 'CONSTRUCTION (15, 16, 17)',                                  # 060-099
 'MANUFACTURING',                                              # 100-399
 'TRANSPORTATION, COMMUNICATIONS, AND OTHER PUBLIC UTILITIES', # 400-499
 'WHOLESALE TRADE',                                            # 500-579
 'RETAIL TRADE',                                               # 580-699
 'FINANCE, INSURANCE, AND REAL ESTATE',                        # 700-720
 'BUSINESS AND REPAIR SERVICES',                               # 721-760
 'PERSONAL SERVICES',                                          # 761-799
 'ENTERTAINMENT AND RECREATION SERVICES',                      # 800-811
 'PROFESSIONAL AND RELATED SERVICES',                          # 812-899
 'PUBLIC ADMINISTRATION',                                      # 900-939
 'ACTIVE DUTY MILITARY',                                       # 940-991
 'EXPERIENCED UNEMPLOYED NOT CLASSIFIED BY INDUSTRY'           # 992-999
]

df_summary_groups = df_groups[df_groups['Job'].apply(lambda x: x in summary_group_list)]

# # remove the summary groups from the group frame
# df_groups = df_groups[df_groups['Job'].apply(lambda x: x not in summary_group_list)]
# df_groups

df_summary_groups

In [ ]:
df_groups.head(10)

In [ ]:
# add in the missing job codes using the lowest cardinality grouping
for i in range(1000):
    code = str(i).zfill(3)
    if df_jobs[df_jobs['Code'] == code].shape[0] ==0:
        # add code to frame with job cat..
        matches = df_groups[(df_groups['Code min'] <= code) & (df_groups['Code max'] >= code)]
        the_job = matches.sort_values('range').iloc[0]['Job']
        df_jobs = df_jobs.append({'Code':code, 'Job':the_job}, ignore_index=True)
df_jobs = df_jobs.sort_values('Code').reset_index().drop('index',axis=1)

In [ ]:
df_jobs.head(35)

In [ ]:
df_summary_groups

In [ ]:
df_jobs

In [ ]:
# define the mappings of code -> Group

# ======= JOBS   ======================
jobs_map = {row['Code']:row['Job'] for i,row in df_jobs.iterrows()}

# ======= SUMMARY GROUPS ==============
summary_group_map = {}
for i in range(1000):
    code = str(i).zfill(3)

    # add code to frame with job cat..
    matches = df_summary_groups[  (df_summary_groups['Code min'] <= code) 
                                & (df_summary_groups['Code max'] >= code)]
    assert matches.shape[0] <= 1
    
    if matches.shape[0] == 0:
        summary_group_map[code] = 'UNKNOWN'
    else:
        summary_group_map[code] = matches['Job'].values[0]

In [ ]:
# apply the maps

df['INDUSTRY_CAT'] = df['INDUSTRY'].apply(lambda x: jobs_map[str(x).zfill(3)])
df['INDUSTRY_SUM'] = df['INDUSTRY'].apply(lambda x: summary_group_map[str(x).zfill(3)])

In [ ]:
# quality checks

print(f"JOB Percentage unknown : {100*sum(df['INDUSTRY_CAT'] =='UNKNOWN') / df.shape[0]:.2f}%")
print(f"SUM Percentage unknown : {100*sum(df['INDUSTRY_SUM'] =='UNKNOWN') / df.shape[0]:.2f}%")

## Place of Birth

### Remove highly imbalanced data

In [ ]:
import seaborn

def _class_balance(df, col):
    df['count'] = 1
    df_plot = df[['count',col]].groupby(col).count().reset_index()

    #define data
    data   = df_plot['count']
    labels = df_plot[col]

    idx = data.argsort()
    data   = [data[i] for i in idx]
    labels = [labels[i] for i in idx]

    #define Seaborn color palette to use
    colors = seaborn.color_palette('pastel')[0:len(data)]

    #create pie chart
    plt.pie(data, labels = labels, colors = colors, autopct='%.1f%%')
    plt.show()

In [ ]:
print(describe_feature('RELAT1'))
print(all_codes['RELAT1'])
_class_balance(df, 'RELAT1')

print(describe_feature('RELAT2'))
print(all_codes['RELAT2'])
_class_balance(df, 'RELAT2')

print(describe_feature('SUBFAM1'))
print(all_codes['SUBFAM1'])
_class_balance(df, 'SUBFAM1')

print(describe_feature('SUBFAM2'))
print(all_codes['SUBFAM2'])
_class_balance(df, 'SUBFAM2')

print(describe_feature('RRELCHLD'))
print(all_codes['RRELCHLD'])
_class_balance(df, 'RRELCHLD')

print(describe_feature('REMPLPAR'))
print(all_codes['REMPLPAR'])
_class_balance(df, 'REMPLPAR')


In [ ]:
df = df.drop(['RELAT2', 'SUBFAM1', 'SUBFAM2', 'RRELCHLD', 'REMPLPAR'], axis=1)

### Remove Low cardinality

With the reduction of records and filtering applied so far; some columns may only contain single values and these will have no use for ML

In [ ]:
# low cardinality suggests not categorical
low_card_cols = []
for name, low_card in (df.nunique(axis=0) <2).items():
    if low_card:
        print(f"{name:<10} -> {describe_feature(name)}")
        low_card_cols += [name]
        
df = df.drop(low_card_cols, axis=1)

### Save the Clean Data

In [ ]:
df.to_csv('us_census_data_cleaned_4.csv')

In [ ]:
df.columns

In [ ]:
df['HOURS'].unique()

In [ ]:
describe_feature('RRELCHLD')

In [ ]:
all_codes['RRELCHLD']

In [ ]:
def combine_rare_classes(df, col, r=0.1, map_to='other'):
    
    gb = df[[col]].copy()
    gb['count'] = 1
    gb = gb[[col, 'count']].copy().groupby(col).sum().reset_index().sort_values(col, ascending=False)
    gb['count'] = 100*gb['count'] / sum(gb['count'])
    
    mapper = {k:map_to for k in gb[gb['count'] < r][col].values}
    
    return mapper, gb

mapper, gb = combine_rare_classes(df, 'RACE', 2.0, map_to = 'Other Race 700 799, 986 999')
df['RACE2'] = df['RACE'].apply(lambda x: mapper.get(x, x))

In [ ]:
mapper, gb = combine_rare_classes(df, 'RACE', 2.0, map_to = 'Other Race 700 799, 986 999')
df['RACE2'] = df['RACE'].apply(lambda x: mapper.get(x, x))

In [ ]:
attr = 'RACE2'
print(describe_feature(attr))
print(all_codes.get(attr,None))
_class_balance(df, attr)

In [ ]:
df[['IMMIGR','PWGT1']].groupby('IMMIGR').sum()

In [ ]:
# create column POB = USA or non-USA
df['POB_T'] = df['POB'].apply(lambda x: all_codes['POB'][x])
df['POB_T'] = df['POB_T'].apply(lambda x: 'United States' if x in states + ['U.S. Territory, Not Specified'] else x)
df.groupby('POB_T').count().sort_values('AGE')['AGE']
df['POB_USA'] = df['POB_T'].apply(lambda x: True if x=='United States' else False)

In [ ]:
df.groupby('POB_USA').count()['AGE']

In [ ]:
df_pob = df.groupby('POB').count().sort_values('AGE', ascending=False).reset_index()
df_pob['POB'] = df_pob['POB'].apply(lambda x: all_codes['POB'][x])
df_pob['POB'] = df_pob['POB'].apply(lambda x: 'United States' if x in states + ['U.S. Territory, Not Specified'] else x)


df_pob.head(25)

In [ ]:
df_pob[df_pob['AGE'] < 10]['POB'].values

In [ ]:
df_pob['POB'].apply(lambda x: 'USA' if x in states else x)

In [ ]:
states = ['Alaska',
 'California',
 'Hawaii',
 'Idaho',
 'Nevada',
 'Oregon',
 'Washington',
 'Arizona',
 'Arkansas',
 'Colorado',
 'Iowa',
 'Kansas',
 'Louisiana',
 'Minnesota',
 'Missouri',
 'Montana',
 'Nebraska',
 'New Mexico',
 'North Dakota',
 'Oklahoma',
 'South Dakota',
 'Texas',
 'Utah',
 'Wyoming',
 'Alabama',
 'Connecticut',
 'Delaware',
 'District of Columbia',
 'Florida',
 'Georgia',
 'Illinois',
 'Indiana',
 'Kentucky',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Mississippi',
 'New Hampshire',
 'New Jersey',
 'New York',
 'North Carolina',
 'Ohio',
 'Pennsylvania',
 'Rhode Island',
 'South Carolina',
 'Tennessee',
 'Vermont',
 'Virginia',
 'West Virginia',
 'Wisconsin']

In [ ]:
for p in df_pob['POB'].unique():
    print(p)

In [ ]:
import pycountry_convert as pc

country_code = pc.country_name_to_country_alpha2("China", cn_name_format="default")
print(country_code)
continent_name = pc.country_alpha2_to_continent_code(country_code)
print(continent_name)

In [ ]:
pc.country_name_to_country_alpha2("China", cn_name_format="default")

In [ ]:
pc.country_mappings.map_countries()['China']['alpha_2']

In [ ]:
pc.country_alpha2_to_continent_code(pc.country_mappings.map_countries()['China']['alpha_2'])

In [ ]:
country_map = pc.country_mappings.map_countries()

map_non_country_to_continent = {
    'Korea, Not Specified'                 : 'AS',
    'Germany, Not Specified'               : 'EU',
    'Union of Soviet Soc.ist Repub.s U.S.' : 'EU',
    'Yugoslavia'                           : 'EU',
    'West Germany'                         : 'EU',
    'East Germany'                         : 'EU',
    'West Berlin'                          : 'EU',
    'Palestine, Not Specified'             : 'AS',
    'West Bank'                            : 'AS',
    'Middle East, Not Specified'           : 'AS',
    'Indochina, Not Specified'             : 'AS',
    'Asia Minor, Not Specified'            : 'AS',
    'Europe, Not Specified'                : 'EU',
    'Central America, Not Specified'       : 'SA',
    'Scotland'                             : 'EU',
    'United Kingdom, Not Specified'        : 'EU',
    'South America, Not Specified'         : 'SA',
    'Northern Ireland'                     : 'EU',
    'Eastern Africa, Not Specified'        : 'AF',
    'Central Africa, Not Specified'        : 'AF',
    'Africa, Not Specified'                : 'AF',
    'North America, Not Specified'         : 'NA',
    'St. Vincent and the Grenadines'       : 'NA',
    'Dominican Repub.'                     : 'NA',
    'Czechoslovakia'                       : 'EU',
    'England'                              : 'EU',
    'Burma'                                : 'AS',
    'Azores Islands'                       : 'EU',
    'Madeira Islands'                      : 'EU',
    'Caribbean, Not Specified'             : 'NA',
    'Asia, Not Specified'                  : 'AS',
    'Western Africa, Not Specified'        : 'AF',
    'Yemen, Peoples Democratic Repub.'     : 'AS',
    'Yemen Arab Repub.'                    : 'AS',
    'Oceania, Not Specified'               : 'OC',
    'British West Indies, Not Specified'   : 'NA',
    'West Indies, Not Specified'           : 'NA',
    'Pitcairn Islands'                     : 'OC',
    'Western Samoa'                        : 'OC',
    'U.S. Virgin Islands'                  : 'NA',
    'Netherlands Antilles'                 : 'NA',
    'Wales'                                : 'EU',
    'St. Kitts Nevis'                      : 'NA',
    'Polynesia, Not Specified'             : 'OC'
}

partial_map = {x:country_map.get(x,{'alpha_2':'Unknown'})['alpha_2'] for x in df_pob['POB'].unique()}


for k,v in partial_map.items():
    if v == 'Unknown':
        partial_map[k] = map_non_country_to_continent.get(k, 'Unknown')
    else:
        try:
            partial_map[k] = pc.country_alpha2_to_continent_code(v)
        except:
            print(k, v)
            partial_map[k] = 'Unknown'


In [ ]:
df['POB_CONTINENT'] = df['POB_T'].apply(lambda x: partial_map[x])

In [ ]:
df.groupby('POB_CONTINENT').count()['AGE']

In [ ]:
map_non_country_to_country = {
    'Korea, Not Specified'                 : 'South Korea',
    
    'Germany, Not Specified'               : 'Germany',
    'West Germany'                         : 'Germany',
    'East Germany'                         : 'Germany',
    'West Berlin'                          : 'Germany',
    
    'Union of Soviet Soc.ist Repub.s U.S.' : 'Russia',
    
    'Palestine, Not Specified'             : 'Palestine',
    'West Bank'                            : 'Palestine',

    'St. Vincent and the Grenadines'       : 'St. Vincent and The Grenadines',
    'St. Kitts Nevis'                      : 'Saint Kitts and Nevis',
    'Dominican Repub.'                     : 'Dominican Republic',
    'Czechoslovakia'                       : 'Czech Republic',

    'Burma'                                : 'Myanmar',
    
    'Azores Islands'                       : 'Portugal',
    'Madeira Islands'                      : 'Portugal',

    'Yemen, Peoples Democratic Repub.'     : 'Yemen',
    'Yemen Arab Repub.'                    : 'Yemen',

    'Pitcairn Islands'                     : 'Pitcairn',
    'Western Samoa'                        : 'Samoa',
    'U.S. Virgin Islands'                  : 'Virgin Islands, U.S.',
    
    
    'Northern Ireland'                     : 'UK',
    'England'                              : 'UK',
    'Wales'                                : 'UK',
    'Scotland'                             : 'UK',
    'United Kingdom, Not Specified'        : 'UK',
    
    
    'Netherlands Antilles'                 : 'Unknown',
    'British West Indies, Not Specified'   : 'Unknown',
    'West Indies, Not Specified'           : 'Unknown',
    'Western Africa, Not Specified'        : 'Unknown',
    'South America, Not Specified'         : 'Unknown',
    'North America, Not Specified'         : 'Unknown',
    'Eastern Africa, Not Specified'        : 'Unknown',
    'Central Africa, Not Specified'        : 'Unknown',
    'Africa, Not Specified'                : 'Unknown',
    'Asia, Not Specified'                  : 'Unknown',
    'Caribbean, Not Specified'             : 'Unknown',
    'Oceania, Not Specified'               : 'Unknown',
    'Yugoslavia'                           : 'Unknown',
    'Middle East, Not Specified'           : 'Unknown',
    'Indochina, Not Specified'             : 'Unknown',
    'Asia Minor, Not Specified'            : 'Unknown',
    'Europe, Not Specified'                : 'Unknown',
    'Central America, Not Specified'       : 'Unknown',
    'Polynesia, Not Specified'             : 'Unknown'
}

partial_map_c = {x:country_map.get(x,{'alpha_2':'Unknown'})['alpha_2'] for x in df_pob['POB'].unique()}
for k,v in partial_map_c.items():
    if v == 'Unknown':
        partial_map_c[k] = country_map.get(map_non_country_to_country.get(k), {'alpha_2':'Unknown'})['alpha_2']

In [ ]:
df['POB_COUNTRY'] = df['POB_T'].apply(lambda x: partial_map_c[x])

In [ ]:
df.groupby('POB_COUNTRY').count().sort_values('AGE', ascending=False).head(25)

In [ ]:
cunts = list(country_map.keys())
cunts.sort()
cunts

In [ ]:
df.columns

In [ ]:
describe_feature('RAGECHLD')

In [ ]:
all_codes['RSPOUSE']

In [ ]:
df.groupby('RAGECHLD').count()

In [ ]:
describe_feature('MIGSTATE')

In [ ]:
df['RSPOUSE'].unique()

In [ ]:
df.groupby('MIGSTATE').count()['AGE']

In [ ]:
all_codes['POWSTATE']

In [ ]:
_class_balance(df, 'RSPOUSE')
_class_balance(df, 'RAGECHLD')

In [ ]:
_class_balance(df, 'MIGSTATE')
_class_balance(df, 'POWSTATE')

In [ ]:
_class_balance(df, 'RPOB')

In [ ]:
# migration
df[df['POWSTATE'] == df["MIGSTATE"]]

In [ ]:
# TODO : Sankey the fuck out of this

df_migration = df[['AGE','POWSTATE',"MIGSTATE"]].groupby(['POWSTATE',"MIGSTATE"]).count().reset_index()

df_migration = df_migration[(df_migration['POWSTATE'] != '00') & (df_migration['MIGSTATE'] != '00')]
# df_migration = df_migration[(df_migration['POWSTATE'] != '99') & (df_migration['MIGSTATE'] != '99')]


df_migration['POWSTATE'] = df_migration['POWSTATE'].apply(lambda x: all_codes['POWSTATE'][x])
df_migration['MIGSTATE'] = df_migration['MIGSTATE'].apply(lambda x: all_codes['MIGSTATE'][x])

df_migration.sort_values('AGE', ascending=False).head(25)

In [ ]:
df_migration['AGE'].sum()

In [ ]:
sum(df['AGE'].astype(int) < 18)

In [ ]:
list(df.columns)

In [ ]:
df['PWGT1'].unique()

In [ ]:
all_codes['PWGT1']

In [ ]:
describe_feature('PWGT1')

In [ ]:
df['count'] = 1

In [ ]:
sum(df['count'] * df['PWGT1'].astype(float))

In [ ]:
df['RPOB']